# Dynamics of Explanation Project

## Stage 4: Data Analysis

This notebook is part of the "Dynamics of Explanation" project and runs the data analysis presented at the 2016 Annual Meeting of the Cognitive Science Society (Paxton, Abney, Castellanos, & Sepulveda, 2016).

To run this notebook, you will need the following files:

* **`./supplementary-code/libraries_and_functions-dyn_exp.r`**: Loads in necessary libraries and creates new functions for our analyses.
* **`./data/final_analysis_data.csv`**: Unified dataframe for all analyses. *Due to ethical considerations relating to participant privacy, no participant data may be shared at this time.*

**Table of Contents:**
1. [Preliminaries](#Preliminaries). Reads in all necessary modules.
1. [Data preparation](#Data-preparation). Imports and prepares data for analysis.


**Written by**: A. Paxton (University of California, Berkeley)   
**Date last modified**: 1 August 2016

***

# Preliminaries

This section reads in all necessary modules.

[To top.](#Dynamics-of-Explanation-Project)

In [1]:
# clear our workspace
rm(list=ls())

In [2]:
# set intial working directory
setwd('./')

In [3]:
# import the source file with all our working directories and custom functions
source('./supplementary-code/libraries_and_functions-dyn_exp.r')

Loading required package: Matrix
Warning message:
: package ‘Matrix’ was built under R version 3.2.4Loading required package: tseriesChaos
Loading required package: deSolve
Warning message:
: package ‘deSolve’ was built under R version 3.2.4
Attaching package: ‘deSolve’

The following object is masked from ‘package:graphics’:

    matplot

Loading required package: fields
Warning message:
: package ‘fields’ was built under R version 3.2.5Loading required package: spam
Loading required package: grid
Spam version 1.3-0 (2015-10-24) is loaded.
Type 'help( Spam)' or 'demo( spam)' for a short introduction 
and overview of this package.
Help for individual functions is also obtained by adding the
suffix '.spam' to the function name, e.g. 'help( chol.spam)'.

Attaching package: ‘spam’

The following objects are masked from ‘package:base’:

    backsolve, forwardsolve

Loading required package: maps
Warning message:
: package ‘maps’ was built under R version 3.2.5Loading required package: plot

***

# Data preparation

This section imports and prepares the data for analysis.

[To top.](#Dynamics-of-Explanation-Project)

***

## Data import

In [4]:
# read in the analysis file
all_data_df2 = data.table::fread('./data/analysis_files/final_analysis_data.csv', sep=',')

Warning message:
In data.table::fread("./analysis_files/final_analysis_data.csv", : Some columns have been read as type 'integer64' but package bit64 isn't loaded. Those columns will display as strange looking floating point data. There is no need to reload the data. Just require(bit64) to obtain the integer64 print method and print the data again.

## Winnow data

Include only saccades and fixations, then remove any participants who don't have at least 100 samples per phase.

In [5]:
# keep only saccades and fixations
all_data_df = all_data_df2 %>%
    dplyr::filter(r_event_info == 2 | r_event_info == 3 )

In [6]:
# identify participants who had at 100 samples in both trials
minimum_slices = all_data_df %>%
    group_by(participant,stimulus) %>%
    summarize(total_slices = n()) %>%
    dplyr::filter(total_slices >= 100) %>%
    dplyr::filter(participant %in% intersect(unique(participant[stimulus==0]),
                                             unique(participant[stimulus==1])))

In [7]:
# remove participants who didn't have enough samples in both trials
all_data_df = dplyr::filter(all_data_df, participant %in% unique(minimum_slices$participant))

## Create plotting dataset

In order to plot correctly, we'll need to create an interim dataset that includes all of the main terms we'll eventually use, without standardizing them.

In [8]:
# create plotting subset
plotting_df = all_data_df %>% ungroup() %>%

    # reset time
    group_by(participant,stimulus) %>%
    mutate(r_time = r_time - min(r_time)) %>%
    mutate(r_time = round(r_time,2)) %>%
    ungroup() %>%
    
    # make sure we don't have multiple pieces of data in the same time now
    group_by(participant,stimulus,r_time) %>%
    arrange(time) %>%
    dplyr::filter(row_number()==1) %>%
    arrange(participant,time) %>%
    ungroup() %>%

    # de-select anything we don't need
    select(-one_of(c('time','trial','timing','frame'))) %>%

    # create pupil change term
    mutate(pupil_change_r = c(0,diff(r_dia_x) / diff(r_time))) %>%
    mutate(pupil_change_l = c(0,diff(l_dia_x) / diff(r_time))) %>%

    # convert all to numeric
    mutate_each(funs(as.numeric))

In [9]:
# save a copy as our standard df name for continuity
all_data_df = plotting_df

# Create first- and second-order polynomials

In [10]:
# create a dataframe with the unique time codes
time_df = data.frame(raw_t = unique(all_data_df$r_time))

In [11]:
# create first- and second-order polynomials for them
t = poly(time_df$raw_t + 1, 2)
time_df[, paste("ot", 1:2, sep="")] = t[time_df$raw_t + 1, 1:2]

In [12]:
# join back to dataframe
all_data_df = left_join(all_data_df,time_df, by = c("r_time" = "raw_t"))

## Center and standardize dynamics dataset

In order to interpret our estimates as effect sizes, we'll need to center and standardize all variables, including the interaction terms (cf. Keith, 2008).

In [13]:
# create analysis subset
all_data_df = all_data_df %>% ungroup() %>%

    # change stimulus so that it's easier to handle with interaction terms
    mutate(stimulus = stimulus - .5) %>%
    
    # create interaction terms
    mutate(stim_change = pupil_change_r * stimulus) %>%
    mutate(stim_pupil = r_dia_x * stimulus) %>%
    mutate(stim_conf = cc_confidence * stimulus) %>%
    mutate(conf_change = cc_confidence * pupil_change_r) %>%
    mutate(conf_pupil = cc_confidence * r_dia_x) %>%
    mutate(stim_sim = stimulus * exp_sim)

In [14]:
# create interaction terms with first-order polynomial
ot1_int = all_data_df %>%
    ungroup() %>%
    mutate_each(funs(. * ot2), -c(r_time,participant,ot1)) %>%
    setNames(paste0('ot1_', names(.)))
ot1_int = plyr::rename(ot1_int, 
                       c("ot1_participant" = "participant", 
                         "ot1_ot1" = "ot1",
                         "ot1_r_time" = "r_time"))

In [15]:
# create interactions with second-order polynomial
ot2_int = all_data_df %>%
    ungroup() %>%
    mutate_each(funs(. * ot2), -c(r_time,participant,ot2)) %>%
    setNames(paste0('ot2_', names(.)))
ot2_int = plyr::rename(ot2_int, 
                       c("ot2_participant" = "participant", 
                         "ot2_ot2" = "ot2",
                         "ot2_r_time" = "r_time"))

In [16]:
# join original df to first-order polynomial interaction term df
all_data_df = merge(all_data_df,ot1_int,by=c("participant"="participant",
                                             "r_time"="r_time",
                                             'ot1'='ot1'))


In [17]:
# join original df to second-order polynomial interaction term df
all_data_df = merge(all_data_df,ot2_int, by=c("participant"="participant",
                                              "r_time"="r_time",
                                              'ot2'="ot2"))

In [18]:
# scale everything
all_data_df = all_data_df %>% ungroup() %>%
    mutate_each(funs(as.numeric(scale(.))))

## Create aggregated dataset

Create an aggregated dataset to explore means and standard deviations, then create separate subsets for the "watch" and "explanation" phases. Standardize each dataset separately.

In [19]:
# get means of pupil size and change
agg_df = all_data_df2 %>%
    dplyr::filter(r_event_info == 2 | r_event_info == 3 ) %>%
    dplyr::filter(participant %in% unique(minimum_slices$participant)) %>%
    mutate(pupil_change_r = c(0,diff(r_dia_x) / diff(r_time))) %>%
    mutate(pupil_change_l = c(0,diff(l_dia_x) / diff(r_time))) %>%
    group_by(participant, stimulus) %>%
    summarize(mean_size_r = as.numeric(mean(r_dia_x)),
              mean_change_r = as.numeric(mean(pupil_change_r)),
              sd_size_r = as.numeric(sd(r_dia_x)),
              sd_change_r = as.numeric(sd(pupil_change_r)),
              cc_exists = unique(cc_exists)[1],
              cc_confidence = unique(cc_confidence)[1],
              exp_sim = unique(exp_sim)[1]) %>%
    mutate(stim_msize = mean_size_r * (stimulus-.5),
           stim_mchange = mean_change_r * (stimulus-.5),
           stim_ssize = sd_size_r * (stimulus-.5),
           stim_schange = sd_change_r * (stimulus-.5))

In [20]:
# isolate and scale explanation data
agg_exp_df = dplyr::filter(agg_df,stimulus==0)
agg_exp_df = data.frame(scale(agg_exp_df))

In [21]:
# isolate and scale watch data
agg_watch_df = dplyr::filter(agg_df,stimulus==1)
agg_watch_df = data.frame(scale(agg_watch_df))

In [22]:
# scale whole aggregated df
agg_df = data.frame(scale(agg_df))

## Summary statistics

In [23]:
length(unique(agg_df$participant))

[1] 25

***

# Data analysis

Let's run the models to explore how the behavioral dynamics are related to outcome.

[To top.](#Dynamics-of-Explanation-Project)

***

## Model 1: Attention and comprehension

Pupil diameter can tap into attention by reflecting working memory load. How much of a participant's eventual semantic similarity can be connected to their changes in pupil diameter over time?

In [127]:
attention_comp = lmer(exp_sim ~ r_dia_x + stimulus + ot1 + ot2 +
                        ot1_r_dia_x + ot1_stimulus +
                        ot2_r_dia_x + ot2_stimulus + 
                        stim_pupil + ot1_ot2 + 
                        ot1_stim_pupil + ot2_stim_pupil + 
                        (1 | cc_confidence), 
                        data=all_data_df)

In [128]:
pander_lme(attention_comp,stats.caption=TRUE)



|        &nbsp;        |  Estimate  |  Std..Error  |  t.value  |     p     |  sig  |
|:--------------------:|:----------:|:------------:|:---------:|:---------:|:-----:|
|   **(Intercept)**    | -0.002546  |   0.01642    |  -0.1551  |  0.8767   |       |
|     **r_dia_x**      | -0.0005053 |   0.01205    | -0.04194  |  0.9665   |       |
|     **stimulus**     |   0.1615   |   0.03922    |   4.117   | 3.835e-05 |  ***  |
|       **ot1**        |   -640.3   |    506.3     |  -1.265   |   0.206   |       |
|       **ot2**        |   -853.3   |    673.7     |  -1.266   |  0.2053   |       |
|   **ot1_r_dia_x**    | -0.0007168 |    0.0122    | -0.05874  |  0.9532   |       |
|   **ot1_stimulus**   |   0.1541   |   0.03915    |   3.937   | 8.241e-05 |  ***  |
|   **ot2_r_dia_x**    | -0.0008471 |    0.0122    | -0.06943  |  0.9446   |       |
|   **ot2_stimulus**   |   0.1587   |   0.03919    |   4.048   | 5.166e-05 |  ***  |
|    **stim_pupil**    |  -0.1649   |   0.03941    |  -4.185   

### Visualize the attention and comprehension results

In [156]:
# generate the "watch" half of the three-way interaction plot
attn_comp_time_watch = ggplot(plotting_df[plotting_df$stimulus==1,], 
                            aes(x=r_time,
                                y=r_dia_x,
                                color=(exp_sim > mean(exp_sim)))) +
                        stat_smooth(method = "lm",
                                    formula = y ~ x + I(x^2),
                                    size = .3,
                                    linetype = 2,
                                    fill = 'grey70') +
                        geom_smooth() + 
                        coord_cartesian(ylim=c(13,17)) +
                        xlab('Time (min)') + ylab('Pupil Diameter (in pixels)') +
                        scale_color_manual(values=c('red3','chartreuse4'),
                                           labels=c('Low','High'),
                                           name = 'Comprehension\n(Similarity)') +
                        theme(legend.position="none") +
                        ggtitle('"Watch" Phase')

In [157]:
# generate the "explanation" half of the three-way interaction plot
attn_comp_time_exp = ggplot(plotting_df[plotting_df$stimulus==0,], 
                            aes(x=r_time,
                                y=r_dia_x,
                                color=(exp_sim > mean(exp_sim)))) +
                        stat_smooth(method = "lm",
                                    formula = y ~ x + I(x^2),
                                    size = .3,
                                    linetype = 2,
                                    fill = 'grey70') +                        
                        geom_smooth() + 
                        coord_cartesian(ylim=c(13,17)) +
                        xlab('Time (min)') + ylab(' ') +
                        scale_color_manual(values=c('red3','chartreuse4'),
                                           labels=c('Low','High'),
                                           name = 'Comprehension\n(Similarity)') +
                        theme(legend.position="none") +
                        ggtitle('"Explanation" Phase')

In [158]:
# create a master legend
master_legend = gtable_filter(ggplot_gtable(
  ggplot_build(attn_comp_time_watch + theme(legend.position="bottom"))), 
  "guide-box")

In [162]:
# assemble the two plots
ggsave('./figures/attn_comp_time-dyn_exp.png',
       units = "in", width = 7, height = 5,
       grid.arrange(
           top=textGrob("Comprehension by Phase\nand Attention over Time",
                        gp=gpar(fontsize=14)),
           attn_comp_time_watch,
           attn_comp_time_exp,
           master_legend,
           ncol = 2,
           heights=c(10, 1)))

<div style="width:500px; height=350px">
![](figures/attn_comp_time-dyn_exp.png)
</div>

**Figure**. Plotting of the three-way interaction effects for phase ("watch" and "explanation"), attention (pupil diameter in pixels), and time on comprehension (LSA-determined similarity between explanation and stimulus). The best-fit quadratic is overlaid in the thin dotted lines of the corresponding color in each panel.

***

## Model 2: Change in attention and comprehension

In [129]:
attn_change_comp = lmer(exp_sim ~ pupil_change_r + stimulus + ot1 + ot2 +
                        ot1_pupil_change_r + ot2_stimulus +
                        ot2_pupil_change_r + ot1_stimulus + 
                        stim_change + ot1_ot2 + 
                        ot1_stim_change + ot2_stim_change + 
                        (1 | cc_confidence), 
                        data=all_data_df)

In [130]:
pander_lme(attn_change_comp,stats.caption=TRUE)



|          &nbsp;          |  Estimate  |  Std..Error  |  t.value  |   p    |  sig  |
|:------------------------:|:----------:|:------------:|:---------:|:------:|:-----:|
|     **(Intercept)**      | -0.005289  |   0.01645    |  -0.3215  | 0.7478 |       |
|    **pupil_change_r**    | -0.002334  |   0.007599   |  -0.3071  | 0.7587 |       |
|       **stimulus**       | -0.001312  |   0.005892   |  -0.2227  | 0.8238 |       |
|         **ot1**          |   -767.1   |    506.8     |  -1.514   | 0.1301 |       |
|         **ot2**          |   -1022    |    674.4     |  -1.515   | 0.1296 |       |
|  **ot1_pupil_change_r**  | -0.002382  |   0.007581   |  -0.3142  | 0.7534 |       |
|     **ot2_stimulus**     | -0.001298  |   0.005867   |  -0.2213  | 0.8249 |       |
|  **ot2_pupil_change_r**  | -0.002377  |   0.007579   |  -0.3136  | 0.7538 |       |
|     **ot1_stimulus**     | -0.001299  |   0.005867   |  -0.2214  | 0.8248 |       |
|     **stim_change**      | 0.0001422  |   0.007623